# SCC.413 Applied Data Mining
# NLP: Week 16
# Twitter Data Collection

## Contents
- [Introduction](#intro)
- [Packages & imports](#packages)
- [Authentication](#authentication)
- [User timelines](#user)
- [Searching for tweets](#searching)
    - [Search operators](#search_ops)
- [Outputting tweets](#outputting)
- [Exercise](#exercise)
- [Further tasks](#tasks)
- [Advanced tasks](#advanced)

<a name="intro"></a>
## Introduction

In this lab exercise you will interact with the [Twitter REST API](https://developer.twitter.com/en/docs) using Python code to download tweets for future analysis. Data collected via APIs are generally much cleaner than web scraped data, and also structured nicely (here, via JSON) for easy querying.

To collect data from Twitter you need to have a Twitter account, and also create an authorised application. If you do not want to do this, you could skip most of this lab and just use pre-collected data, although it is useful to see how to collect your own data. One option would be to partner with a neighbour and use a single Twitter account. As a minimum, you should observe how you can read in previously collected Twitter data, and output this in a different format (see [Outputting tweets](#outputting), and [Exercise](#exercise)).

Ensure you have downloaded the code from the git repository (as described on Moodle), and place it in a folder for this lab. Your h-drive is probably the best place, although keep an eye on the space available with the various data files you will be creating in the lab.

The code provides a functions collecting Twitter data via the [Twitter REST API](https://developer.twitter.com/en/docs). We use the [Twython](https://github.com/ryanmcgrath/twython) Python package to assist us, although others are available, most notably [Tweepy](https://github.com/tweepy/tweepy).

Ensure you have completed the separate instructions for creating a developer account and Twitter app.

<a name="packages"></a>
## Packages & Imports

The Twython package will need installing on Google Colab, as below. Non-standard packages are also included in `requirements.txt`, if you need to install them on your own machine.

In [ ]:
!pip install twython

You should upload all of the provided files to a Google Drive folder, you can then access these files from your Python code. See also the files tab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


The below code adds a folder from Google Drive. You may need to edit the path to match your own.

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/413/wk16')

This particular lab uses the Twython package. Other imports are included in one cell here for convenience.

To interact with the Twitter API, you need developer credentials, with a the *Consumer Key (API Key)*, *Consumer Secret (API Secret)*, *Access Token*, and *Access Token Secret*. These should be copied and saved into the relevant variables in `twitter_auth.py` before running the cells below. The authorisation variables are read in with the import below. You can replace the import and add the variables directly here (though this is not good practice, as it reveals your credentials).

In [ ]:
from twython import Twython, TwythonError, TwythonRateLimitError #https://github.com/ryanmcgrath/twython
import sys
import time
import json

from twitter_auth import *

<a name="authentication"></a>
## Authentication

Our hook into the Twitter API is via [Twython](https://github.com/ryanmcgrath/twython), and we make API calls via functions on a Twitter authenticated Twython object. We create and authorise this below, with supplied credentials (from `twitter_auth.py`).

In [ ]:
twitter = Twython(consumer_key, consumer_secret, access_token, access_secret)

<a name="user"></a>
## User timelines

The Twitter API allows for the downloading of any (unprotected) user's tweets, limited to their last 3,200. Collecting a user's tweets can be useful for various research questions, such as comparing language usage across individuals/organisations, and for performing various authorship analysis tasks (as we'll see later in the module).

A function is provided below for collecting a given user's Tweets. Review the code and check your understanding of how it is collecting tweets. The function is also available from `user_tweets.py`.

The Twitter API throttles the downloading of data, here allowing for 200 tweets per request, and 1,500 requests per 15-minute window (and 100,000 requests per day). Therefore we need to collect 200 tweets at a time, with an older starting point each time, until there are no more tweets available. If we hit the [rate limit](https://developer.twitter.com/en/docs/basics/rate-limiting), we pause the collection until the rate-limit window resets. Other options are available for the collecting the user tweets: [user timeline](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline.html). How would you discard tweets which are replies to other users?

In [ ]:
def get_user_tweets(twitter, screen_name, **kwargs):
    """uses twitter (Twython object) to collect tweets from user with screen_name (no @), can include extra search parameters for get_user_timeline, returns list of tweets"""
    
    #initialize a list to hold all the tweets
    user_tweets = []
    try:
        #make initial request for most recent tweets (200 is the maximum allowed count).
        #We normally don't want retweets, so we set include_rts to false.
        #tweet_mode="extended" allows for full text tweets, rather than truncated (i.e. over 140 chars)
        #https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline.html
        new_tweets = twitter.get_user_timeline(screen_name=screen_name,count=200,include_rts=False,tweet_mode="extended", **kwargs)

        #add to list
        user_tweets.extend(new_tweets)

        #save the id of the oldest tweet less one, this is the starting point for collecting further tweets.
        oldest = user_tweets[-1]['id'] - 1
        #keep grabbing tweets until there are no tweets left to grab. Twitter limits us to 3,200 (including retweets)
        while len(new_tweets) > 0:
            try:
                #all subsequent requests use the max_id param to prevent duplicates
                new_tweets = twitter.get_user_timeline(screen_name=screen_name,count=200,include_rts=False,tweet_mode="extended",max_id=oldest, **kwargs)
                user_tweets.extend(new_tweets)
                oldest = user_tweets[-1]['id'] - 1
                print("...%s tweets downloaded so far" % (len(user_tweets)))
            except TwythonRateLimitError as e:
                #We have hit the rate limit, so we need to take a break.
                #find how much time need to sleep for.
                remainder = float(twitter.get_lastfunction_header(header='x-rate-limit-reset')) - time.time()
                print("sleeping for %d seconds" % remainder)
                #Pause until we can go again.
                time.sleep(remainder)
                continue
                
    except TwythonRateLimitError as e:
        #We have hit the rate limit on first call, so we need to take a break, and start again.
        #find how much time need to sleep for.
        remainder = float(twitter.get_lastfunction_header(header='x-rate-limit-reset')) - time.time()
        print("sleeping for %d seconds" % remainder)
        #Pause until we can go again.
        time.sleep(remainder)
        #start again
        return get_user_tweets(twitter, screen_name, **kwargs)

    except TwythonError as e:
        print(e)

    return user_tweets

To collect tweets for a user, we simply call the method, providing our authenticated Twython object (twitter), and a twitter user screen name (without the @). Below we collect [Lancaster University's twitter timeline](https://twitter.com/LancasterUni).

In [ ]:
tweets = get_user_tweets(twitter, "LancasterUni")

...376 tweets downloaded so far
...566 tweets downloaded so far
...761 tweets downloaded so far
...946 tweets downloaded so far
...1125 tweets downloaded so far
...1312 tweets downloaded so far
...1483 tweets downloaded so far
...1670 tweets downloaded so far
...1858 tweets downloaded so far
...2055 tweets downloaded so far
...2255 tweets downloaded so far
...2448 tweets downloaded so far
...2628 tweets downloaded so far
...2819 tweets downloaded so far
...2996 tweets downloaded so far
...3047 tweets downloaded so far
...3047 tweets downloaded so far


This returns a list of tweet dictionary objects. A lot of information is provided per Tweet. The attributes are detailed for [Tweet objects in the API](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object). Review the information that is available.

The first tweet will be the latest tweet. You can see the list of keys, and a full tweet below. We can also view individual attributes, such as the tweet text.

In [ ]:
print(tweets[0].keys())

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])


In [ ]:
print(tweets[0])

{'created_at': 'Tue Feb 22 13:44:01 +0000 2022', 'id': 1496118581712637961, 'id_str': '1496118581712637961', 'full_text': 'Using smartphone apps could reveal your identity, according to research by our Dr Heather Shaw (@H_Shawberry)📱🔍\n\n@PsychScience @lancspsychres @LancsUniSciTech\n\nhttps://t.co/KTq6c5EwsJ', 'truncated': False, 'display_text_range': [0, 183], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'H_Shawberry', 'name': 'Dr Heather Shaw', 'id': 87503878, 'id_str': '87503878', 'indices': [96, 108]}, {'screen_name': 'PsychScience', 'name': 'Association for Psychological Science', 'id': 19078387, 'id_str': '19078387', 'indices': [113, 126]}, {'screen_name': 'lancspsychres', 'name': 'Lancaster Psychology', 'id': 195781766, 'id_str': '195781766', 'indices': [127, 141]}, {'screen_name': 'LancsUniSciTech', 'name': 'Science & Technology at Lancaster', 'id': 25527873, 'id_str': '25527873', 'indices': [142, 158]}], 'urls': [{'url': 'https://t.co/KTq6c5Ew

In [ ]:
tweets = get_user_tweets(twitter, "UCREL_NLP")
print(tweets[0]['full_text'])

...108 tweets downloaded so far
...139 tweets downloaded so far
...168 tweets downloaded so far
...240 tweets downloaded so far
...240 tweets downloaded so far
9/9 And we have more papers coming up next week @CL2021IE #CL2021 along with our colleagues in @CorpusSocialSci See you there! #NLProc #corpuslinguistics  https://t.co/iicYfmE1YB


<a name="searching"></a>
## Searching for tweets

The Twitter API also allows for the searching for Tweets, albeit only over a sample from the last 7 days (unless you pay). This could be useful for collecting a selection of tweets on specific topic, or mentioning people.

A function is provided below for performing searches, in a similar manner to extracting a user’s tweets. Review and check your understanding of the code. The function is also available from `search_tweets.py`.

In [ ]:
def search_twitter(twitter, search_term, limit, **kwargs):
    """uses twitter (Twython object) to collect tweets returned from given search_term, up to limit, , can include extra search parameters, returns list of tweets"""
    
    #initialise list of tweets
    tweets = []

    try:
        #count=100 is the maximum allowed
        #tweet_mode="extended" allows for full text tweets, rather than truncated (i.e. over 140 chars)
        #https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets.html
        search_results = twitter.search(q=search_term,tweet_mode="extended",count=100, **kwargs)
        tweets.extend(search_results['statuses'])

        if len(tweets) == 0:
            print("No results found")
            return tweets
        
        #save the id of the oldest tweet less one, this is the starting point for collecting further tweets.
        oldest = tweets[-1]['id'] - 1
        #keep grabbing tweets until there are no tweets left to grab.
        while len(search_results['statuses']) > 0 and len(tweets) < limit:
            try:
                #all subsequent requests use the max_id param to prevent duplicates
                search_results = twitter.search(q=search_term,tweet_mode="extended",count=100,max_id=oldest, **kwargs)
                tweets.extend(search_results['statuses'])
                oldest = tweets[-1]['id'] - 1
                print("...%s tweets downloaded so far" % (len(tweets)))
            except TwythonRateLimitError as e:
                #We have hit the rate limit, so we need to take a break.
                #find how much time need to sleep for.
                remainder = float(twitter.get_lastfunction_header(header='x-rate-limit-reset')) - time.time()
                print("sleeping for %d seconds" % remainder)
                #Pause until we can go again.
                time.sleep(remainder)
                continue

    except TwythonRateLimitError as e:
        #We have hit the rate limit on first call, so we need to take a break, and start again.
        #find how much time need to sleep for.
        remainder = float(twitter.get_lastfunction_header(header='x-rate-limit-reset')) - time.time()
        print("sleeping for %d seconds" % remainder)
        #Pause until we can go again.
        time.sleep(remainder)
        #start again
        return search_twitter(twitter, search_term, limit, **kwargs)
                
    except TwythonError as e:
        print(e)

    return tweets[:limit]

To search, simply provide a search string and a limit of the number of tweets to return, e.g. to get 500 tweets with the hashtag #NLProc (common hashtag for NLP related stuff):

In [ ]:
tweets = search_twitter(twitter, "#NLProc", 500)

...175 tweets downloaded so far
...267 tweets downloaded so far
...367 tweets downloaded so far
...467 tweets downloaded so far
...557 tweets downloaded so far


In [ ]:
print(tweets[0]["full_text"])

The three PhD positions on Multilingual Low-Resource #NLProc funded by @Carlsbergfondet are now up! The project offers a lot of freedom in terms of research direction, and the conditions offered when doing a PhD in Denmark are hard to beat. Apply here: https://t.co/4FJdwS4A0A https://t.co/U48q4QUuNI


<a name="search_ops"></a>
### Search operators

The are a number of search operators available, allowing for quite complex searches: [search operators](https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators). Ignore the instructions on URL encoding the search string, *Twython* takes care of this for us.

Search strings can be built up with multiple parameters. For example, to search for tweets from the *@LancasterUni* account mentioning *research*:

In [ ]:
tweets = search_twitter(twitter, "from:LancasterUni research", 10)
for tweet in tweets:
    print("----")
    print(tweet['full_text'])

...3 tweets downloaded so far
----
Using smartphone apps could reveal your identity, according to research by our Dr Heather Shaw (@H_Shawberry)📱🔍

@PsychScience @lancspsychres @LancsUniSciTech

https://t.co/KTq6c5EwsJ
----
RT @LancasterPress: AI generated faces are MORE trustworthy than real faces says research co-authored by Dr Sophie Nightingale @sjnightinga…
----
Drugs to increase insulin signaling may be effective for treating autism, according to research by @BLSLancasterUni's Dr Neil Dawson (@neilneuro)

@RoyalSociety @LancasterUniFHM 

https://t.co/7cilAGNKLa


There are also different parameters available for the search request itself: [search parameters](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets.html).

These can be passed through the search_twitter method (using [kwargs](https://book.pythontips.com/en/latest/args_and_kwargs.html)). For example, to restrict a search to only tweets in English:

In [ ]:
tweets = search_twitter(twitter, "\"faux pas\"", 10, lang='en')
for tweet in tweets:
    print("----")
    print(tweet['full_text'])

----
RT @Thuggedraccoon: Microwaving fish is an office faux pas. I roast mine over an open flame in the bathroom
----
In the night light, do you still feel your pain?
For the valor you waited, it never came
If you were able, would you go change the past,
To mend a faux pas with one last chance?
----
Yet another major IKN faux pas in visiting Russia at a very wrong time.
Blind support by a few senior ex-FO officers puzzling.
Wonder what rationale FO gave.
The visit will further annoy US &amp; its NATO allies.  
#PDM ought to condemn IKN as Parliament has not been consulted.
----
The Russia/Ukraine situation may represent Davos’s first faux pas. He simply can’t be beaten by any measure, which accounts for why every WEF Western power is dead set on bringing him down. The fact is ‘Putin is their only stumbling block to totalitarian world domination.
----
@bachus89 @MichealMartinTD Yes his visit to Hempel was a huge blunder. However Dev was held in very high regard by the Jewish community i

Try this with "fr", and without specifying the language to see the difference.

<a name="outputting"></a>
## Outputting tweets

To create a corpus for later use, you may want to save tweets to a file. A series of functions are provided below to output to JSON, plain text, and also to read in JSON saved tweets. Review and check your understanding of these functions. These are also provided in `tweets_json.py`.

In [ ]:
def to_full_json(tweets, filepath="tweets.json"):
    """Saves to filepath with the provided tweets with all attributes, in JSON."""
    with open(filepath, 'w') as f:
        #Dump json file. indent=4 prints the output prettier, but will increase disk space.
        json.dump(tweets, f, indent=4)

def to_minimal_json(tweets, filepath="tweets.json"):
    #This reduces each tweet to the set of keys (attributes) listed.
    #Other attributes can be used here, see https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object
    atts=['id_str', 'full_text']
    minimal_tweets = [{k:tweet[k] for k in atts} for tweet in tweets]
    with open(filepath, 'w') as f:
        #Dump json file. indent=4 prints the output prettier, but will increase disk space.
        json.dump(minimal_tweets, f, indent=4)

def to_just_text(tweets, filepath="tweets.txt"):
    """Saves to filepath with the provided tweets in plaintext, one tweet per line"""
    with open(filepath, 'w') as f:
        for tweet in tweets:
            #Linebreaks are replaced so we have one tweet per line.
            f.write("{}\n".format(tweet['full_text'].replace("\n", "  ").replace("\r", "  ")))
            
def load_json_tweets(filepath):
    """Loads a JSON file into a list of tweet dictionary objects"""
    tweets = json.load(open(filepath))
    return tweets

The full JSON can be saved, although note that this will take up some space. You can save to minimal JSON, and plain text by using the different functions.

In [ ]:
to_minimal_json(tweets, "@LancasterUni-min.json")

To load in previously saved tweets, just use load_json_tweets, as below, to load in the provided UCREL tweets.

In [ ]:
ucrel_tweets = load_json_tweets("@UCREL_NLP-full.json")
print(ucrel_tweets[0])

{'created_at': 'Fri Aug 02 18:45:50 +0000 2019', 'id': 1157361882489065473, 'id_str': '1157361882489065473', 'full_text': "And that's all folks. #acl2019nlp is over. It's been a great week, thanks to all the @ACL2019_Italy conference and workshop organisers. Goodbye Florence from the @UCREL_Lancaster #NLProc team https://t.co/lHhSARIKdW", 'truncated': False, 'display_text_range': [0, 215], 'entities': {'hashtags': [{'text': 'acl2019nlp', 'indices': [22, 33]}, {'text': 'NLProc', 'indices': [179, 186]}], 'symbols': [], 'user_mentions': [{'screen_name': 'ACL2019_Italy', 'name': 'ACL2019', 'id': 1035524448319885312, 'id_str': '1035524448319885312', 'indices': [85, 99]}, {'screen_name': 'UCREL_Lancaster', 'name': 'UCRELResearchCentre', 'id': 1423596631, 'id_str': '1423596631', 'indices': [162, 178]}], 'urls': [{'url': 'https://t.co/lHhSARIKdW', 'expanded_url': 'https://twitter.com/DocElhaj/status/1156618784175734784?s=20', 'display_url': 'twitter.com/DocElhaj/statu…', 'indices': [192, 215]}

<a name="exercise"></a>
## Exercise

Using the [list of attributes available](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object), produce a list of tweets (in JSON format or plain text, e.g. separated by a tab (TSV)) containing the time the tweet was written and the text. Of course, there are numerous ways the tweets could be outputted, e.g. into a database or a CSV file. Feel free to experiment with different outputs that might be useful to you. You can use the provided UCREL_NLP tweets(`@UCREL_NLP-full.json`) for this, or any other collection of tweets you have made.

In [ ]:
# Exercise 1

def to_tsv(tweets, filepath="tweets.tsv"):
    """Saves to filepath with the provided tweets in plaintext, one tweet per line"""
    with open(filepath, 'w') as f:
        for tweet in tweets:
            #Linebreaks are replaced so we have one tweet per line.
            f.write("{}\t{}\n".format(tweet['created_at'], tweet['full_text'].replace("\n", "  ").replace("\r", "  ")))

ucrel_tweets = load_json_tweets("@UCREL_NLP-full.json")
to_tsv(ucrel_tweets, "ucrel.tsv")


<a name="tasks"></a>
## Further Tasks

Come up with your own searches, and discuss with your neighbors. A few you can try:

1. Find tweets mentioning *paper* and the *#NLProc* hashtag, which are not retweets.
2. Find 10 positive tweets about *rain*, and 10 negative tweets about *rain*.
3. Find Tweets to or from *@LancasterUni* mentioning *storm*, *wet* or *rain*.
4. Find recent tweets mentioning *snow*.
5. Find tweets mentioning *rain* from the Lancaster area.

Also, think about what useful tweet attributes are available to output for the above searches.

In [ ]:
# 1. Find tweets mentioning paper and the #NLProc hashtag, which are not retweets.

tweets = search_twitter(twitter, "#NLProc paper -filter:retweets", 5)
for tweet in tweets:
    print("----")
    print(tweet['full_text'])

## note multiple search terms are always AND. use `OR` for either, or, both.

----
Does anyone know when LREC paper notification will be? The website still says "tbd". https://t.co/Ep5gqZH6uf #NLProc
----
Kindly consider submitting your paper to the workshop on Language Technology and Resources for a Fair, Inclusive, and Safe Society at LREC. #LATERAISSE2022 #LREC2022 #NLProc #Bias #Equality #DiversityandInclusion
----
Having seen all these user-unfriendly paper submission systems, I'm just loving easychair more! Why NLP conferences don't go back to it?! #NLProc #easychair #torturedbyOpenreview
----
Great news! Recent @MetaAI work cited our work. It’s time to rearrange this work to submit #NLProc 🤨🙏🤟 check our paper here https://t.co/CjJjrgQOUo https://t.co/EKTXue7yWX
----
Don't have enough local language data?Leverage augmentation to improve your #NLProc performance. Paper "Improving Short Text Classification through Global Augmentation Methods" preprint https://t.co/uSXGL8G6JJ and library https://t.co/7IiGj3ECkE
#MotherLanguageDay #AfricanNLP
6/n


In [ ]:
#  2. Find 10 positive tweets about rain, and 10 negative tweets about rain.

tweets = search_twitter(twitter, "rain :)", 10)
for tweet in tweets:
    print("----")
    print(tweet['full_text'])
    
tweets = search_twitter(twitter, "rain :(", 10)
for tweet in tweets:
    print("----")
    print(tweet['full_text'])

# note, not convinced what Twitter is actually doing here with positive/negative, 
#seems to just return where an emoticon is used. Could use sentiment analysis as an alternative.

----
RT @_kness: 🦧I often get asked how I do it and what I use and are commissions open and all sorts of things. I love answering questions - es…
----
@Sefayildirim33 Biz de merakla bekliyoruz Sefa, bakalım gelecek günlerde neler olacak. :)
----
@Sefayildirim33 Adı üstünde diyorsun, Win yani :)
----
@jaempwiss00 hi rain, lets be moots! :D help rt like my pinned, ty! 😼
----
@Sefayildirim33 Ama en azından 1 tanesinin yeri senin için ayrıdır? :)
----
الان فقط سریال Something in the rain ، آهنگش و خود rain جوابه :)
----
RT @kiioll100: 스티커 수량조사!!!폼 (입금폼 아님XXX) ~2/27까지
https://t.co/qSDFhbgh4F

수량조사량+a 만큼 미리 제작 후 판매할 예정이여요
꼭 구매하실 분만 참여부탁드려요!! 감사합니다 :D https://…
----
Mam jebanych spredawczykow w klasie s tym jeden typ to ten z którym byłam w przyjacielskiej relacji w tamtym roku :))))
----
@AlphaTrilogy @DianaJunakovic This is beautiful! Loved the rain vibe! :)
----
@Sefayildirim33 Bugünün kuruyla 57.800 $ - 65.000 $ arasında olabilir diyorsun yani? :)
----
RT @BTS_7roses: Indian ARMys dil me 

In [ ]:
# 3. Find Tweets to or from @LancasterUni mentioning *storm, *wet* or *rain*.

tweets = search_twitter(twitter, "from:LancasterUni OR to:LancasterUni AND storm OR wet OR rain", 10)
for tweet in tweets:
    print("----")
    print(tweet['full_text'])

#note use of OR, ORs are done together, then AND'd with the from:LancasterUni
# might not have any results, it did when I first did it! Could try different words, e.g. research

...4 tweets downloaded so far
----
@ianbakersson Looks wild, wet, and windy! Hope you all had a good reunion 👋 ^David
----
@LancasterUni Sorry to hear about the damage done by the storm I hope no-one was hurt.  Just wondering if your offer holder open day still on tomorrow?
----
Update: Edward Roberts Court remains closed for the time being but all sections of the perimeter road have now reopened and campus remains open and operational. Please remain vigilant on campus and at home today during Storm Eunice
----
⚠️ Storm Eunice update

Due to the wind affecting roofs on campus, Edward Roberts Court and a section of our perimeter road are currently closed.

ℹ️ Our staff and students can find out more including any further updates on their Student Portal or Staff Intranet.


In [ ]:
# 4. Find recent tweets mentioning snow.

tweets = search_twitter(twitter, "snow", 10, result_type="recent")
for tweet in tweets:
    print("----")
    print(tweet['full_text'])
    
# this is a search parameter, rather an operator in the search query string. Compare with "popular"
# this is equivalent to 'latest' and 'top' on Twitter itself.

----
RT @RYOOO1515: めっちゃ面白いし楽しいブラザービート🤣
色々ツッコミどころがあるSnow Manのパフォーマンス！とっても好きだーー
#ブラザービート
#テレ東音楽祭
----
Snow Man割とガッチガチに踊る曲とダンスが多かったから、これだけ洒落の効いた曲でだけど、しめるとこ締めて踊るの本当に“良い”んだよなぁ。遊びを聞かせた中にSnow Manの良さがしっかり滲み出てるのが好きだなぁ。
----
RT @teretoongakusai: 🌸🌸🥳#テレ東音楽祭 2022春🌸🌸
＼テレビ東京系 ただいま放送中❣️／

Snow Man さんから
コメントいただきました♪
このあと、まもなく登場です👏

#SnowMan
#おうちでテレ東音楽祭
----
一旦深呼吸しよう、呼吸して呼吸
----
Snow blind idiot out without her glasses.
----
Snow Manブラザービート最高だった！！
スーツみんな似合ってるしスタイルみんな良すぎだしみんな可愛かった
阿部くんの髪型好きだなーおでこかわいい
というか翔太もふっかもでこ出しスタイルが好きです
----
RT @im_usa99b: ブラザービートでパニパニしてたらスノマニビルさんで2万切ってるー😳
Snow Man毎回ナイスタイミングでミリオン行くよね…これはもしかして？
オリさんでも売れてるし！
シングルHELLO HELLO、Secret Touch、KISSIN’ MY…
----
Snow Man「反転メガネ王決定戦」全てが逆さまって…ムリ～!! https://t.co/lwAYKqhgD5 @YouTubeより
----
RT @ongaku_tv2nd: テレビ初披露

映画『おそ松さん』主題歌
♪　ブラザービート ／ Snow Man　①

テレ東音楽祭
#スノブラザー https://t.co/DYPXbqjkqd
----
RT @SMC_Srinagar: Ongoing snow clearance in Islamyabal ward. https://t.co/naqBlrge1F


In [ ]:
# 5. Find tweets mentioning rain from the Lancaster area.

tweets = search_twitter(twitter, "rain", 10, geocode="54,-2.7,10mi")
for tweet in tweets:
    print("----")
    print(tweet['full_text'])
    
# again, this is a search parameter, rather than part of search query string.
# latitude, longitude, radius.
# may need to set radius larger to get results.

----
Rain or shine (but mostly rain), @ucu is here! 

#OneOfUsAllOfUs #USSmess #USSmess https://t.co/6gRPDvqHNu
----
@rossi_tg Could do with a little bit of sun too. We have rain and floods over here in Manchester.
----
COURSE UPDATE - OPEN. Heritage 9 open. All on temporary greens. Another 8mm rain last night takes the total past 200mm for February, with over a week remaining. @JHorrocksPGA @Breightmetgc
----
@GaryBarlow @The_Lowry Enjoy hometown G ❤🎹🎤
&amp; stay safe in this wind &amp; rain!
☔🌪🌀🌬☔🌫☔🌬🌀🌪☔ https://t.co/fWsUYAk5EK
----
To help you stay safe behind the wheel – truck or car – read up on these top tips on road safety: https://t.co/I1MfRwhZ9k
You’ll find tips on:
👁️ Daily walk round
❄️ Driving in rain, fog, snow, ice
✋ Leaving space
💡 Lights
🛁 Clean car
🩹 Essentials to carry https://t.co/LHlJsvm1Kj
----
Just literally had to throw the cat outside (gently, obvs!), as she's spent the weekend staring daggers at me for causing wind and rain...
----
COURSE STATUS CLOSED. Another 

<a name="advanced"></a>
## Optional advanced tasks


The Twitter API provides access to further information about tweets, users, and plenty more. Here’s a list of things you can try if you have time. Please feel free to make other suggestions.

1. Many other methods are available from Twython linked to Twitter API requests: https://twython.readthedocs.io/en/latest/api.html. One potentially useful task you should be able perform by adapting the available code is to collect the user details of a given account (see *show_user*).
2. Expanding on 1., you could collect a list of users (e.g. a user's followers) and then collect all of their user information and tweets.
3. You can use the Twitter Streaming API to collect tweets in real-time as they are posted. See the instructions for implementing this with Twython: https://twython.readthedocs.io/en/latest/usage/streaming_api.html, and attempt to collect all tweets mentioning a word of interest.

In [ ]:
# Advanced 1. Many other methods are available from Twython linked to Twitter API requests: https://twython.readthedocs.io/en/latest/api.html. One potentially useful task you should be able perform by adapting the available code is to collect the user details of a given account (see show_user).

twitter.show_user(screen_name="LancasterUni")

{'contributors_enabled': False,
 'created_at': 'Fri Mar 20 15:35:26 +0000 2009',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'We’re a highly ranked collegiate university with a global reputation for teaching and research.',
 'entities': {'description': {'urls': []},
  'url': {'urls': [{'display_url': 'lancaster.ac.uk',
     'expanded_url': 'http://www.lancaster.ac.uk',
     'indices': [0, 23],
     'url': 'https://t.co/tiy4EYH1rT'}]}},
 'favourites_count': 15931,
 'follow_request_sent': False,
 'followers_count': 65056,
 'following': False,
 'friends_count': 1182,
 'geo_enabled': True,
 'has_extended_profile': False,
 'id': 25521930,
 'id_str': '25521930',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': None,
 'listed_count': 778,
 'location': 'Lancaster, England',
 'name': 'Lancaster University',
 'notifications': False,
 'profile_background_color': 'FFFFFF',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1

In [ ]:
# Advanced 2 Expanding on 1., you could collect a list of users (e.g. a user's followers) and then collect all of their user information and tweets.

following = twitter.get_friends_list(screen_name="LancasterUni")
followers = twitter.get_followers_list(screen_name="LancasterUni")

for f in following['users'][:10]:
    print(f['screen_name'])
    
# could easily adapt above to get tweets too, using existing function above.

LancasterBlack
PETRASiot
MattBradbury_
_MichaelStead
ProfWHollmann
LUSoM_
social_lu
UBC
LancasterNorth
UnlockArchives


In [ ]:
# Advanced 3 You can use the Twitter Streaming API to collect tweets in real-time as they are posted. See the instructions for implementing this with Twython: https://twython.readthedocs.io/en/latest/usage/streaming_api.html, and attempt to collect all tweets mentioning a word of interest.

from twython import TwythonStreamer

class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        if 'extended_tweet' in data:
            text = data['extended_tweet']['full_text']
        else:
            text = data['text']

        print("-----")
        print(text)

    def on_error(self, status_code, data):
        print(status_code)
        
stream = MyStreamer(consumer_key, consumer_secret, access_token, access_secret)
stream.statuses.filter(track="party")


-----
@NBCNews CCP (Chinese Communist party)Propaganda and spy’s is in the America boompoint_down_tone1flag_lr
这段视频解释了中共间谍单伟健如何渗透到美国金融体系中，通过向中共公司筹集和投资数千亿美元...
#shanweijian 
#ccpspy 
#ccpliedpeopledied 
#pensionfunds
https://t.co/m63qXafz4R
-----
RT @xxaroundyou: รอรคือจัดหมวดห่าอะไรชื่อเป็น lovely room, party room คือใช้คำธรรมดาก็ได้ไหม sfw, nsfw, romance, comedy, thriller, horror…
-----
RT @_john_gell: If Conservative MPs are in receipt of Russian money and the Conservative Party itself is dependant on Russian donations, ho…
-----
RT @BarrieCunning: Great to see  @Bill_Esterson interviewed by the wonderful @SophiaSleigh. The message is loud and clear the @UKLabour par…
-----
RT @ZionG38274521: I'm not really a birthday enthusiast, but Laycon's jam, New Dimension, is inspiring me to throw a party next month. That…
-----
@ScotTories anything to say about your party funding stream? How safe is Faslane if you're in Russia's pockets?
-----
RT @justinmadders: The point was well made- Russia

KeyboardInterrupt: ignored